In [326]:
from astropy.io import ascii
import numpy as np
import gurobipy as gp
from gurobipy import *
Placa_madre = ascii.read("Opti - Placa Madre (1).dat")
Memorias = ascii.read("Opti - RAM (2).dat")
Almacenamiento = ascii.read("Opti - Almacenamiento SSD (3).dat")
Procesador = ascii.read("Opti - Procesador (4).dat")
Fuente_de_poder = ascii.read("Opti - Fuente de poder (5).dat")
Tarjeta_de_video = ascii.read("Opti - Tarjeta de video (6).dat")

Componentes = [Placa_madre, Memorias, Almacenamiento, Procesador, Fuente_de_poder, Tarjeta_de_video]
tiendas = ['Las Condes','Manuel Montt','Mall Arauco Maipu','Mall Plaza Alameda','Chillan']

I_ = range(0, 6)
M_ = [range(0, len(Componentes[0])),range(0, len(Componentes[1])),range(0, len(Componentes[2])),
     range(0, len(Componentes[3])),range(0, len(Componentes[4])),range(0, len(Componentes[5]))]
T_ = range(0, 5)

c = {(i,m,t) : Componentes[i]['Precio ($)'][m] for i in I_ for m in M_[i] for t in T_}
k = {t : 1 for t in T_}
p = {(i,m): Componentes[i]['Potencia (W)'][m] for i in [1,3,4,5] for m in M_[i]}
e = {(i,m): Componentes[i]['Especificacion'][m] for i in [1,2,3,5] for m in M_[i]}
E = {1: 16, 2: 500, 3: 28.8, 5: 8}
r = {m: Componentes[0]['Slots de memoria'][m] for m in M_[0]}
a = {m: Componentes[0]['Almacenamiento'][m] for m in M_[0]}
S = {(i,m,t): Componentes[i][tiendas[t]][m] for i in I_ for m in M_[i] for t in T_}

R = {(1,m): [i for i in range(len(Componentes[1]['Precio ($)'])) if 
             Componentes[1]['Tipo de memoria'][i] != Componentes[0]['Tipo de memoria'][m] or
             Componentes[1]['Frecuencia (MHz)'][i] < Componentes[0]['Velocidad minima (MHz)'][m] or
             Componentes[1]['Frecuencia (MHz)'][i] > Componentes[0]['Velocidad maxima (MHz)'][m]   
            ]
             for m in M_[0]}

for m in M_[0]:
    R[3,m] = [i for i in range(len(Componentes[3]['Precio ($)'])) if 
             Componentes[3]['Marca'][i] != Componentes[0]['Plataforma'][m]]
    
H = {(1,m): [i for i in range(len(Componentes[1]['Precio ($)'])) if 
             Componentes[1]['Frecuencia (MHz)'][i] > Componentes[3]['Velocidad maxima (MHz)'][m] or
             Componentes[1]['Tipo de memoria'][i] != Componentes[3]['Tipos de memoria'][m] + ' DIMM']
             for m in M_[3]}

phi = {t: 1 for t in T_}


l = []
for i in I_:
    for m in M_[i]:
        for t in T_:
            l.append((i,m,t))
            

In [327]:
# MODELO
model = Model("Costo_Mínimo_Computador")
            
# -------------------------------------------
# DEFINICIÓN DE VARIABLES
x = model.addVars(l,vtype = GRB.INTEGER, name = "Componente", lb = 0.0)
y = model.addVars(T_,vtype = GRB.BINARY, name = "envio")
z = model.addVars(T_,vtype = GRB.BINARY, name = "envio_gratis")

# -------------------------------------------
# RESTRICCIONES

# Se debe comprar solo una unidad de algunos componentes
model.addConstrs((quicksum(quicksum(x[i,m,t] for m in M_[i]) for t in T_) == 1 for i in [0,3,4]), name = "Componentes_unitarios")

# Se debe comprar a lo mas una tarjeta de video
model.addConstrs((quicksum(quicksum(x[i,m,t] for m in M_[i]) for t in T_) <= 1 for i in [5]), name = "Componentes_unitarios")

# Respetar el limite de espacios de RAM en la placa madre
model.addConstr((quicksum(quicksum(x[1,m,t] for m in M_[1]) for t in T_) <= quicksum(quicksum(r[m] * x[0,m,t] for m in M_[0]) for t in T_)), name = "Respetar_RAM")

# Respetar el limite de espacios de almacenamiento en la placa madre
model.addConstr((quicksum(quicksum(x[2,m,t] for m in M_[2]) for t in T_) <= quicksum(quicksum(a[m] * x[0,m,t] for m in M_[0]) for t in T_)), name = "Respetar_almacenamiento")

# Especificaciones
model.addConstrs((quicksum(quicksum(e[i,m] * x[i,m,t] for m in M_[i]) for t in T_) >= E[i] for i in [1,2,3,5]), name = "Especificaciones_minimas")

# Respetar el limite de potencia
model.addConstr((quicksum(quicksum(quicksum(p[i,m] * x[i,m,t] for m in M_[i]) for t in T_) for i in [1,3,4,5]) >= 0), name = "Respetar_limite_potencia")

# Cobrar envio
A = 1e10
model.addConstrs((quicksum(quicksum(x[i,m,t] for m in M_[i]) for i in I_) <= A * y[t] for t in T_), name = "cobrar_envio")

# envio grtis
model.addConstrs((quicksum(quicksum(c[i,m,t] * x[i,m,t] for m in M_[i]) for i in I_) >= phi[t] * z[t] for t in T_), name = "Envio_gratis")

# compatibilidad placa madre con otros componentes
model.addConstrs((A*(1 - quicksum(x[0,m,t] for t in T_)) >= quicksum(x[i,k,t] for t in T_) for m in M_[0] for i in [1,3] for k in R[i,m]), name = "Compatibilidad_placa_madre")

# compatibilidad procesador con otros componentes
model.addConstrs((A*(1 - quicksum(x[3,m,t] for t in T_)) >= quicksum(x[i,k,t] for t in T_) for m in M_[3] for i in [1] for k in H[i,m]), name = "Compatibilidad_procesador")

# Respetar disponibilidad en tienda
model.addConstrs((x[i,m,t] <= S[i,m,t] for i in I_ for m in M_[i] for t in T_), name = 'Stock')


# Naturaleza variables
model.addConstrs((x[i,m,t] >= 0 for i in I_ for m in M_[i] for t in T_),name = "Naturaleza_x_it")
model.addConstrs((y[t] >= 0 for t in T_),name = "Naturaleza_y_it")                 
model.addConstrs((z[t] >= 0 for t in T_),name = "Naturaleza_z_ijt")  
# como ya fue definido antes que z era variable binaria, es necesario poner su restriccion de naturaleza?
#model.addConstrs((z[i,t]=?),name = "Naturaleza z_it")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [328]:
obj = quicksum((y[t] - z[t])*k[t]  + quicksum(quicksum(c[i,m,t] * x[i,m,t] for m in M_[i]) for i in I_) for t in T_)
                 
model.setObjective(obj, GRB.MINIMIZE)
                 

# Actualizar modelo
model.update()
       
# Escribir formulación
model.write("Modelo {}.lp".format(model.ModelName)) 

# Optimizar
model.optimize()



#################################

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 1910 rows, 655 columns and 10285 nonzeros
Model fingerprint: 0xb13e3a41
Variable types: 0 continuous, 655 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+10]
  Objective range  [1e+00, 5e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1892 rows and 583 columns
Presolve time: 0.02s
Presolved: 18 rows, 72 columns, 266 nonzeros
Variable types: 0 continuous, 72 integer (42 binary)
Found heuristic solution: objective 1098970.0000
Found heuristic solution: objective 1088470.0000
Found heuristic solution: objective 964520.00000

Root relaxation: objective 9.329253e+05, 31 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [329]:
# Mostrar resultados
print(f"\n### Ganancia óptima: {model.objVal} calculado en {model.runtime} segundos. ###\n")


### Ganancia óptima: 932930.0 calculado en 0.07695770263671875 segundos. ###



In [330]:
model.printAttr("x") # all non-zero solution values


    Variable            x 
-------------------------
Componente[0,10,2]            1 
Componente[1,10,0]            1 
Componente[1,12,0]            1 
Componente[2,14,0]            1 
Componente[3,19,3]            1 
Componente[4,0,4]            1 
Componente[5,12,1]            1 
    envio[0]            1 
    envio[1]            1 
    envio[2]            1 
    envio[3]            1 
    envio[4]            1 
envio_gratis[0]            1 
envio_gratis[1]            1 
envio_gratis[2]            1 
envio_gratis[3]            1 
envio_gratis[4]            1 


In [331]:
for i in I_:
    for m in M_[i]:
        for t in T_:
            if x[i,m,t].x>0:
                print(Componentes[i]['ID'][m])
                if i in [1,2,3,5]:
                    print(Componentes[i]['Especificacion'][m])

38613
37040
8
32318
8
29235
500
37819
31.2
25576
37351
8
